# HMS - Harmful Brain Activity Classification
## Preprocess and plot with MNE (for Human)

In [ ]:
import numpy as np
import pandas as pd
from functools import cache
from typing import Literal
import matplotlib.pyplot as plt

BASE_PATH = '/kaggle/input/hms-harmful-brain-activity-classification'

train_df = pd.read_csv(f'{BASE_PATH}/train.csv')
train_df.head()

@cache
def load_eeg(eeg_id, data_type:Literal['train','test']='train'):
    eeg_df = pd.read_parquet(f'{BASE_PATH}/{data_type}_eegs/{eeg_id}.parquet')
    return eeg_df

In [ ]:
train_df.head()

In [ ]:
eeg_df = load_eeg(1628180742)
eeg_df

## MNE
https://mne.tools/stable/index.html

In [ ]:
import mne

In [ ]:
info = mne.create_info(
    eeg_df.columns.to_list(),
    ch_types=(["eeg"]*(len(eeg_df.columns)-1))+['ecg'],
    sfreq=200
)
info.set_montage("standard_1020")

In [ ]:
raw = mne.io.RawArray(
    eeg_df.to_numpy().T*1e-6,    # µV to V
    info
)

In [ ]:
mne.set_config('MNE_BROWSE_RAW_SIZE','16,8')
raw.plot(start=20, duration=10)
plt.show()

- [learningeeg.com](https://www.learningeeg.com/montages-and-technical-components#filters) suggests these filters:
    - Highpass: 1Hz
    - Lowpass: 70Hz
    - Notch: 60Hz (50Hz for Europe)

In [ ]:
raw_filtered = raw.copy().filter(l_freq=1, h_freq=70,).notch_filter(60, picks='eeg')
raw_filtered.plot(start=20, duration=10)
plt.show()

In [ ]:
bipolar = [
    ['Fp1', 'F7'], ['F7', 'T3'], ['T3', 'T5'], ['T5', 'O1'],    # Left Temporal
    ['Fp2', 'F8'], ['F8', 'T4'], ['T4', 'T6'], ['T6', 'O2'],    # Right Temporal
    ['Fp1', 'F3'], ['F3', 'C3'], ['C3', 'P3'], ['P3', 'O1'],    # Left Parasagittal
    ['Fp2', 'F4'], ['F4', 'C4'], ['C4', 'P4'], ['P4', 'O2'],    # Right Parasagittal
    ['Fz', 'Cz'], ['Cz', 'Pz'],   # Central
]

anode, cathode = list(map(list,zip(*bipolar)))

raw_bip_ref = mne.set_bipolar_reference(raw_filtered, anode=anode, cathode=cathode)
raw_bip_ref.plot(start=20, duration=10)
plt.show()

In [ ]:
raw_bip_ref.time_as_index(1)

## Utility

In summary, you can copy and use the following cell.

In [ ]:
from functools import cache
from typing import Literal
import numpy as np
import pandas as pd

import mne

mne.set_config('MNE_BROWSE_RAW_SIZE','16,8')

@cache
def load_eeg(eeg_id, data_type:Literal['train','test']='train'):
    BASE_PATH = '/kaggle/input/hms-harmful-brain-activity-classification'
    eeg_df = pd.read_parquet(f'{BASE_PATH}/{data_type}_eegs/{eeg_id}.parquet')
    return eeg_df

@cache
def get_filtered_bipolar(eeg_id, data_type:Literal['train','test']='train'):
    bipolar = [
        ['Fp1', 'F7'], ['F7', 'T3'], ['T3', 'T5'], ['T5', 'O1'],    # Left Temporal
        ['Fp2', 'F8'], ['F8', 'T4'], ['T4', 'T6'], ['T6', 'O2'],    # Right Temporal
        ['Fp1', 'F3'], ['F3', 'C3'], ['C3', 'P3'], ['P3', 'O1'],    # Left Parasagittal
        ['Fp2', 'F4'], ['F4', 'C4'], ['C4', 'P4'], ['P4', 'O2'],    # Right Parasagittal
        ['Fz', 'Cz'], ['Cz', 'Pz'],   # Central
    ]
    anode, cathode = list(map(list,zip(*bipolar)))

    eeg_df = load_eeg(eeg_id, data_type)
    
    info = mne.create_info(
        eeg_df.columns.to_list(),
        ch_types=(["eeg"]*(len(eeg_df.columns)-1))+['ecg'],
        sfreq=200
    )
    info.set_montage("standard_1020")
    
    raw = mne.io.RawArray(
        eeg_df.to_numpy().T*1e-6,    # µV to V
        info
    ).filter(l_freq=1, h_freq=70,).notch_filter(60, picks='eeg')
    
    return mne.set_bipolar_reference(raw, anode=anode, cathode=cathode)

def bipolar(eeg_id, offset=0., duration=10.0, data_type:Literal['train','test']='train', plot=True):
    bip_ref = get_filtered_bipolar(eeg_id, data_type)
    
    start = 25.0 + offset - duration/2
    if plot:
        bip_ref.plot(start=start, duration=duration)
    
    start_idx = bip_ref.time_as_index(start).item()
    stop_idx = bip_ref.time_as_index(start + duration).item()
    return bip_ref.get_data(start=start_idx, stop=stop_idx)

## Examples

In [ ]:
data = bipolar(1628180742, 24, 10)
data.shape

In [ ]:
data = bipolar(1628180742, 6, 10)
data.shape